In [1]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa 
import design_functions
from v_p_chapa_cabeca.v_p_chapa_cabeca import *
from draw_autocad.draw_autocad_figures import *
from materials import * 

In [2]:
perfil_escolhido = W_610x140_0


aço_escolhido = ASTM_A36

solda_escolhida =E70XX

perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)


##### Cálculo da disposição da chapa, dos parafusos e do diâmetro minimo

In [3]:
M = 856000 #kN.mm 

V = 480 #kN

T = 720 #kN

parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =1)
diametros = diametros_A325

S = dim_chapa_parafuso(M,V,T,perfil_escolhido,disposicoes_gerdau_chapa_cabeca,parafuso,diametros,gamma_a2)

chapa = S[3]
ver_parafuso = S[4]

N = len(ver_parafuso)  #Número total de parafusos
n = (ver_parafuso["x (mm)"] == ver_parafuso["x (mm)"].iloc[0]).sum()  #número de parafusos por coluna
n_p_c = N/n  #número de parafusos por camada

S[1]

'1.1/8'

In [6]:
chapa.df

,vértice,x (mm),y (mm)
0,1,0.000,0
1,2,337.175,0
2,3,337.175,717
3,4,0.000,717
4,5,0.000,0


##### Cálculo da espessura da chapa de cabeça

In [7]:
rigida = 1

F_r_total = resistencia_total(parafuso,gamma_a2)

F_t_Sd=solicitante_parafuso_tração(M,chapa.B, np.unique(ver_parafuso["y (mm)"]),n_p_c, parafuso , n, S[0]) + T/(n_p_c*n)

exp = exp_placa(aço_escolhido, chapa, espessuras, rigida, ver_parafuso, pol_to_mm(S[1]), F_r_total,F_t_Sd,gamma_a1)

exp = pol_to_mm(exp)

exp

25.4

##### Cálculo da espessura do filete de solda na alma e na mesa

In [19]:
espessura= espessura_solda(M,V,T,solda_escolhida,perfil_escolhido,exp,True,gamma)

esp = math.ceil(espessura) #mm

##### Abertura do projeto no pyautocad

In [9]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing11.dwg


In [10]:
limpar_desenho(acad)

Desenho completamente limpo!


In [11]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

In [12]:
# Chamando a função para desenhar a chapa 3D
objetos_chapa = criar_chapa_3d(acad, chapa.df, exp)

In [13]:
objetos_parafusos=[]
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]

    # Adicionar circunferência no ponto
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,exp), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,0), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    # Transladar hexágono para o ponto atual
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, y_centro, exp)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(*hexagono_transladado[j])
        p2 = APoint(*hexagono_transladado[j + 1])
        obj = acad.model.AddLine(p1, p2)
        objetos_parafusos.append(obj)

In [14]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, (chapa.B / 2) - (perfil_escolhido.b_f / 2), posicao_y=20, altura_z=exp)

In [15]:
# Rotacionar apenas a seção do perfil:
for obj in objetos_parafusos:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_chapa:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,-perfil_escolhido.b_f/2,0  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_chapa:
    obj.Move(APoint(0,0,0),vetor)


for obj in objetos_parafusos:
    obj.Move(APoint(0,0,0),vetor)

In [20]:
escrever_descricao(acad,0,0,max(chapa.df["y (mm)"])+10 ,"Chapa","",perfil_escolhido.nome,esp)